<a href="https://colab.research.google.com/github/gastan81/generative-ai/blob/main/0_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot

## 1. Settings

Installation

In [1]:
%%bash
pip install -qqq -U faiss-cpu
#!pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U langchain-huggingface
pip install -qqq -U pypdf
pip install -qqq -U streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
%%bash
pip install -qqq -U jupyter
pip install -qqq -U ipywidgets

Libraries

In [2]:
import os
#from google.colab import userdata
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

Colab: token

In [ ]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

Local: token

In [ ]:
# token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

Model

In [3]:
# hf_model = 'mistralai/Mistral-7B-Instruct-v0.3'
# hf_model = 'microsoft/Phi-3.5-mini-instruct'
# hf_model = 'microsoft/phi-4'
# hf_model = 'Qwen/Qwen2.5-7B-Instruct'
# hf_model = 'mistralai/Mistral-Nemo-Instruct-2407'

hf_model = 'microsoft/phi-4-gguf'
llm = HuggingFaceEndpoint(repo_id=hf_model)

This will not be needed when everything on github:

https://drive.google.com/file/d/14PqeK1VNokE5PZ-b-hVLqV-h4xnSR7B4/view?usp=sharing Alice in Wonderland

https://drive.google.com/file/d/1mJIwux2e0XvZUDBoEj9rb3OlHlYsTxXb/view?usp=sharing Statistics in Python

https://drive.google.com/file/d/1i5ZhAxtIown7RzH1Sh13y2eqcXKvsMyq/view?usp=sharing Statistics in R

Embedding

In [4]:
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "data/cache/"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model
    , cache_folder=embeddings_folder)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Create vector from loaded document

In [6]:
# # The document
# # file_id = '1mJIwux2e0XvZUDBoEj9rb3OlHlYsTxXb' # Google drive file's ID
# # file = 'https://drive.google.com/uc?export=download&id=' + file_id
# file = 'data/The CIA World Factbook 2018-2019.pdf'

In [ ]:
# # Read pages
# loader = PyPDFLoader(file)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [8]:
# # Split text
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
# docs = text_splitter.split_documents(pages)

In [ ]:
# # Review the loaded document
# print(f"{docs[10].metadata}\n")
# print(docs[10].page_content)

In [ ]:
# Create vector
vector_db = FAISS.from_documents(docs, embeddings)

In [ ]:
# Save vector
vector_db.save_local("data/CIA_faiss_index")

Load vector from saved index

In [5]:
vector_db = FAISS.load_local("data/CIA_faiss_index", embeddings, allow_dangerous_deserialization=True)

RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/faiss/faiss/impl/io.cpp:67: Error: 'f' failed: could not open data/CIA_faiss_index/index.faiss for reading: No such file or directory

Retriever

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

Chat setup

In [ ]:
template = """You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation.
Keep your answers short, succinct, informative, and clear, so that the couterpart can learn from you.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:
"""

# chat_history = []

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

doc_retriever = create_history_aware_retriever(
    llm, retriever, prompt
)

doc_chain = create_stuff_documents_chain(llm, prompt)

Chatbot

In [ ]:
chain = create_retrieval_chain(
    doc_retriever, doc_chain
)
history = []
# Start the conversation loop
while True:
  user_input = input("\nYou: ")

  # Check for exit condition
  if user_input.lower() == 'end':
      print("Ending the conversation. Goodbye!")
      break

  # Get the response from the conversation chain
  response = chain.invoke({"input":user_input, "chat_history": history, "context": retriever})
  history.extend([{"role": "human", "content": response["input"]},{"role": "assistant", "content":response["answer"]}])
  # Print the chatbot's response
  print(response["answer"])


Response: Environmental issues and international agreements related to conservation, deforestation, desertification, freshwater resources, climate change, biodiversity, and pollution.
Ending the conversation. Goodbye!


Streamlit

In [ ]:
%%writefile data/streamlit_app.py

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.vectorstores import FAISS, faiss-cpu
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st

# llm
hf_model = 'mistralai/Mistral-7B-Instruct-v0.3' # 'microsoft/Phi-3.5-mini-instruct'
llm = HuggingFaceEndpoint(repo_id=hf_model)

# embeddings
embedding_model = 'sentence-transformers/all-MiniLM-l6-v2'
embeddings_folder = 'data/cache/'

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model
    , cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local('data/CIA_faiss_index', embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# prompt
template = """You are a nice chatbot having a conversation with a human.
Answer the question based only on the following context and previous conversation.
Keep your answers short, succinct, informative, and clear, so that the couterpart can learn from you.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

prompt = ChatPromptTemplate.from_messages([
    ('system', template),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
])

# bot with memory
@st.cache_resource
def init_bot():
    doc_retriever = create_history_aware_retriever(llm, retriever, prompt)
    doc_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(doc_retriever, doc_chain)

rag_bot = init_bot()


##### streamlit #####

st.title('CIA World Factbook 2018-2019')

# Initialise chat history
# Chat history saves the previous messages to be displayed
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])

# React to user input
if prompt := st.chat_input('Curious minds wanted!'):

    # Display user message in chat message container
    st.chat_message('human').markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({'role': 'human', 'content': prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner('Asking CIA...'):

        # send question to chain to get answer
        answer = rag_bot.invoke({'input': prompt, 'chat_history': st.session_state.messages, 'context': retriever})

        # extract answer from dictionary returned by chain
        response = answer['answer']

        # Display chatbot response in chat message container
        with st.chat_message('assistant'):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({'role': 'assistant', 'content':  response})

Writing data/streamlit_app.py
